In [1]:
import numpy as np
import awkward as ak
import glob
import dask_awkward as dak
from distributed import LocalCluster, Client, progress

client =  Client(n_workers=60,  threads_per_worker=2, processes=True, memory_limit='8 GiB') 


In [14]:
# year = "2017"
year = "2016postVFP"
# load_path = "/depot/cms/users/yun79/hmm/copperheadV1clean/test_test//stage1_output/2018/f1_0/cutflow_data_*"
load_path = f"/depot/cms/users/yun79/hmm/copperheadV1clean/cutflow_18Apr2025_singleMuTrigMatch//stage1_output/{year}/f1_0/cutflow_*"
filelist = glob.glob(f"{load_path}/*.parquet")
ak_zip=dak.from_parquet(filelist)

In [15]:
ak_zip.fields

['TotalEntries',
 'LumiMaskMetFilterPv',
 'HLT_filter',
 'muon_base_selection',
 'muon_trig_match',
 'electron_veto',
 'Jet_selection_njetsLeq2',
 'HemVeto_filter',
 'anti_ttH_btag_cut',
 'ggH_cut',
 'signal_fit_region']

In [16]:
# # sig_region_yield = [ak_zip[field][:50] for field in ak_zip.fields]
# sig_region_yield = [ak_zip[field] for field in ak_zip.fields]
# # sig_region_yield = ak.prod(sig_region_yield)

In [17]:
sig_region_arr = ak.ones_like(ak_zip.TotalEntries)

for field in ak_zip.fields:
    # if "HemVeto" in field:
        # continue
    arr = ak_zip[field]
    sig_region_arr = sig_region_arr & arr
    print(f"{field} sum: {ak.sum(sig_region_arr).compute()}")

# sig_region_arr = ak.num(sig_region_arr, axis=0).compute()
sig_region_arr = ak.sum(sig_region_arr).compute()

TotalEntries sum: 331976208
LumiMaskMetFilterPv sum: 326911885
HLT_filter sum: 216656712
muon_base_selection sum: 12873562
muon_trig_match sum: 12361998
electron_veto sum: 12358288
Jet_selection_njetsLeq2 sum: 11890536
HemVeto_filter sum: 11890536
anti_ttH_btag_cut sum: 11780450
ggH_cut sum: 11719595
signal_fit_region sum: 188430


In [63]:
# Rereco

year = "2018"
# year = "2016postVFP"
# load_path = "/depot/cms/users/yun79/hmm/copperheadV1clean/test_test//stage1_output/2018/f1_0/cutflow_data_*"
# load_path = f"/depot/cms/users/yun79/hmm/copperheadV1clean/cutflow_rereco_yun_Apr20//stage1_output/{year}/data_B"
load_path = f"/depot/cms/users/yun79/hmm/copperheadV1clean/test2//stage1_output/{year}/data_D"
filelist = glob.glob(f"{load_path}/*.parquet")
ak_zip=dak.from_parquet(filelist)

In [64]:
ak_zip.fields

['TotalEntries',
 'LumiMaskMetFilterPv',
 'HLT_filter',
 'muon_base_selection',
 'muon_trig_match',
 'electron_veto',
 'Jet_selection_njetsLeq2',
 'anti_ttH_btag_cut',
 'ggH_cut',
 'signal_fit_region',
 'njets_nominal',
 'jet1_pt_nominal',
 'jj_mass_nominal',
 'jj_dEta_nominal',
 'nBtagLoose_nominal',
 'nBtagMedium_nominal',
 'dataset',
 'year']

In [69]:
sig_region_arr = ak.ones_like(ak_zip.TotalEntries)

fields2compute = [
    'TotalEntries',
 'LumiMaskMetFilterPv',
 'HLT_filter',
 'muon_base_selection',
 'muon_trig_match',
 'electron_veto',
 # 'Jet_selection_njetsLeq2',
 'anti_ttH_btag_cut',
 'ggH_cut',
 'signal_fit_region',
]

for field in ak_zip.fields:
    # print(field)
    # if "ggH_cut" in field:
    #     continue
    # if "anti_ttH_btag_cut" in field:
    #     continue
    # if "Jet_selection_njetsLeq2" in field:
        # continue
    if field not in fields2compute:
        continue
    arr = ak_zip[field]
    # print(f"{field} arr: {arr.compute()}")
    sig_region_arr = sig_region_arr & arr
    # print(f"{field} sum: {ak.sum(sig_region_arr).compute()}")
print(f"{field} sum: {ak.sum(sig_region_arr).compute()}")

# sig_region_arr = ak.num(sig_region_arr, axis=0).compute()
# sig_region_arr = ak.sum(sig_region_arr).compute()

year sum: 382375


In [60]:
ak_zip["jet1_pt_nominal"][sig_region_arr].compute()
# ak_zip["njets_nominal"][sig_region_arr].compute()
# ak.max(ak_zip["jet1_pt_nominal"][sig_region_arr]).compute()

<Array [35.3, 25.6, 0, 40.4, 0, 32.5, 41.2, 0, 0, 0] type='10 * ?float32'>

In [62]:
# print(ak_zip["ggH_cut"][sig_region_arr].compute())
print(ak_zip["jet1_pt_nominal"][sig_region_arr].compute())

print(ak_zip["jj_mass_nominal"][sig_region_arr].compute())
print(ak_zip["jj_dEta_nominal"][sig_region_arr].compute())
print(ak_zip["nBtagLoose_nominal"][sig_region_arr].compute())
print(ak_zip["nBtagMedium_nominal"][sig_region_arr].compute())
print(ak_zip["njets_nominal"][sig_region_arr].compute())


[35.3, 25.6, 0, 40.4, 0, 32.5, 41.2, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 1, 0, 1, 0, 1, 1, 0, 0, 0]


In [25]:
ak_zip["njets_nominal"][sig_region_arr&ak_zip.Jet_selection_njetsLeq2].compute()


<Array [2, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0] type='11 * ?float64'>

In [5]:
ak.sum(ak_zip["signal_fit_region"]).compute()

np.int64(102765)